## Section 0. Libraries and Packages

In [2]:
# pip install tensorflow
# !pip install tensorflow_federated

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.5/558.5 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━

In [1]:
import tensorflow as tf
import tensorflow_federated as tff

## Section 1. MNIST data loading from the Keras Datasets.

In [44]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

## Section. Defining model a architecture.

This code defines a function called `create_model()` that returns a simple convolutional neural network (CNN) model for image classification.

The model architecture includes the following layers:

`Reshape`: This layer changes the input shape of the image from (28, 28, 1) to (28, 28, 1).

`Conv2D`: This is a 2D convolutional layer with 32 filters and a kernel size of (3, 3). It applies the ReLU activation function to the output of the convolution operation.

`MaxPooling2D`: This is a 2D max pooling layer with a pool size of (2, 2).

`Flatten`: This layer flattens the output of the previous layer into a 1D array.

`Dense`: This is a fully connected layer with 10 neurons, which corresponds to the 10 classes in the MNIST dataset. It applies the softmax activation function to the output of the layer.

`The create_model()` function returns the model object, which can be used to create a Federated Learning model using the from_keras_model() method from the TensorFlow Federated library.

In [45]:
def create_model():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Reshape(input_shape=(28,28,1), target_shape=(28,28,1)),
      tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  return model

## Section Creation of Federated Learning model.

We’ll create a Federated Learning model using the TFF library:

This code snippet defines a function `create_federated_model()` that creates a Federated Learning model using the TFF library. The `from_keras_model()` function from TFF is used to convert the Keras model created in the previous step into a Federated Learning model.

The keras_model argument specifies the Keras model to be converted, which in this case is the model returned by `create_model()`. The input_spec argument specifies the shape and data type of the input data. In this example, the input data has a shape of (None, 28, 28, 1) and a data type of tf.float32, which represents the grayscale images in the MNIST dataset.

In [46]:
def create_model():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Reshape(input_shape=(28,28,1), target_shape=(28,28,1)),
      tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  return model

## Section. Definition the Federated Learning process.


This code defines two functions related to the Federated Learning process: federated_train_data and federated_train.

In [49]:
federated_train_data = [[x_train[i:i+100], y_train[i:i+100]] for i in range(0, len(x_train), 100)]
federated_train_data = [tf.data.Dataset.from_tensor_slices((x, y)).batch(20) for x, y in federated_train_data]

def federated_train(model, federated_train_data):
    return tff.learning.build_federated_averaging_process(
        model_fn=lambda: model,
        client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
        server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0)
    )(federated_train_data)

## Section. Training the Federated Learning model

In [50]:
iterative_process = federated_train(create_federated_model(), federated_train_data)
for round_num in range(10):
  state = iterative_process.next(state)
  print('Round {}: loss={}'.format(round_num, state['loss']))

TypeError: ignored